In [1]:
import csv
import json
import os
import pprint
from dictor import dictor
import pandas as pd

import requests

In [14]:
headers = {'content-type': 'application/json', 'x-hasura-admin-secret': 'V3ntur4Pr0dH4sur4Tr4v3lC0re2o2o'}

In [15]:
url = 'https://core.venturatravel.org/v1/graphql'

In [16]:
query = """
{
  service {
    name
    duration_in_h
    expense_type
    payment_type
    get_feedback
    bed_distribution
    service_cost_period {
      valid_from
      valid_until
      contract_url
      single_surcharge
      single_room_cost_per_pax
      double_room_cost_per_pax
      amount_of_xx_pax_to_pay_tc_or_driver_costs
      tc_driver_costs_xx_pax
      tc_driver_costs_xx_plus_1_pax
      tc_cost_per_day
      tc_cost_per_departure
      cost_pax_1
      cost_pax_2
      cost_pax_3
      cost_pax_4
      cost_pax_5
      cost_pax_6
      cost_pax_7
      cost_pax_8
      cost_pax_9
      cost_pax_10
      cost_pax_11
      cost_pax_12
      cost_pax_13
      cost_pax_14
      cost_pax_15
      cost_pax_16
      currency {
        id
        abbreviation
        exchange_rate_to_eur
      }
    }
    provider {
      name
      code
      provider_level {
        name
      }
      provider_type {
        name
      }
    }
    trip_version_days {
      day_number
      trip_version {
        version_name
        trip {
          project {
            name
          }
          code
          name
          trip_type {
            name
          }
          seo {
            locale_id
          }
        }
      }
    }
    trip_version_day_services {
      trip_version_day {
        day_number
        trip_version {
          version_name
          trip {
            project {
              name
            }
            code
            name
            trip_type {
              name
            }
            seo {
              locale_id
            }
          }
        }
      }
    }
    departure_tc_1_services {
      departures {
        trip_version {
          version_name
          trip {
            project {
              name
            }
            code
            name
            trip_type {
              name
            }
            seo {
              locale_id
            }
          }
        }
      }
    }
  }
}
"""

In [17]:
# submit the request
r = requests.post(url=url, json={'query': query}, headers=headers)

In [18]:
services = r.json()['data']['service']

In [19]:
headers = [
        'service_name',
        'provider_name',
        'provider_code',
        'trip_code', # trip dependent
        'provider_type',
        'service_level',
        'duration_in_h',
        'expense_type',
        'payment_type',
        'get_feedback',
        'bed_distribution',
        'contract_link',
        'trip_version_name', # trip dependent
        'market',# trip dependent
        'trip_type',# trip dependent
        'brand',# trip dependent
        'trip_day',# trip dependent
        'currency',
        'cost_pax_1',
        'cost_pax_2',
        'cost_pax_3',
        'cost_pax_4',
        'cost_pax_5',
        'cost_pax_6',
        'cost_pax_7',
        'cost_pax_8',
        'cost_pax_9',
        'cost_pax_10',
        'cost_pax_11',
        'cost_pax_12',
        'cost_pax_13',
        'cost_pax_14',
        'cost_pax_15',
        'cost_pax_16',
        'cost_eur_pax_1',
        'cost_eur_pax_2',
        'cost_eur_pax_3',
        'cost_eur_pax_4',
        'cost_eur_pax_5',
        'cost_eur_pax_6',
        'cost_eur_pax_7',
        'cost_eur_pax_8',
        'cost_eur_pax_9',
        'cost_eur_pax_10',
        'cost_eur_pax_11',
        'cost_eur_pax_12',
        'cost_eur_pax_13',
        'cost_eur_pax_14',
        'cost_eur_pax_15',
        'cost_eur_pax_16',
]

In [20]:
rows = []
for obj in services:
    acc_days = obj.get('trip_version_days', []) # accomodation where the service is used
    tc_days = obj.get('departure_tc_1_services', []) # TC where the service is being used
    day_services = obj.get('trip_version_day_services', []) # other services used
    
    # skip services not being used 
    if len(acc_days) == 0 and len(tc_days) == 0 and len(day_services) == 0:
        continue;
    
    # common values
    base_obj = {'service_name': dictor(obj, 'name'),
            'provider_name': dictor(obj, 'provider.name'),
            'provider_code': dictor(obj,'provider.code'),
            'provider_type': dictor(obj,'provider.provider_type.name'),
            'service_level': dictor(obj, 'provider.provider_level.name'),
            'duration_in_h': dictor(obj, 'duration_in_h'),
            'expense_type': dictor(obj, 'expense_type'),
            'payment_type': dictor(obj, 'payment_type'),
            'get_feedback': dictor(obj, 'get_feedback'),
            'bed_distribution': dictor(obj, 'bed_distribution')}

    # find correct service cost period.
    cost_periods = obj.get('service_cost_period', [])
    cost_period = {};
    if (len(cost_periods) == 1):
        cost_period = cost_periods[0];
    elif (len(cost_periods) > 1):
        cost_period = cost_periods[0];

    # fill out the values with the coversion to eur
    for i in range(1, 17):
        costPax = cost_period.get("cost_pax_{}".format(i))
        base_obj["cost_pax_{}".format(i)] = costPax
        exchangeRate = dictor(cost_period, 'currency.exchange_rate_to_eur')
        
        
        if costPax != None and exchangeRate != None: 
            base_obj["cost_eur_pax_{}".format(i)] = costPax * exchangeRate;

    base_obj["currency"] = dictor(cost_period, 'currency.abbreviation')
    base_obj["contract_link"] = dictor(cost_period, 'contract_url')
    
    if len(acc_days) > 0:
        base_obj['trip_code'] = dictor(acc_days, '0.trip_version.trip.code')
        base_obj['trip_version_name'] = dictor(acc_days, '0.trip_version.version_name')
        base_obj['market'] = dictor(acc_days, '0.trip_version.trip.seo.locale_id')
        base_obj['trip_type'] = dictor(acc_days, '0.trip_version.trip.trip_type.name')
        base_obj['brand'] = dictor(acc_days, '0.trip_version.trip.project.name')
        base_obj['trip_day'] = dictor(acc_days, '0.day_number')
    elif len(tc_days) > 0:
        base_obj['trip_code'] = dictor(tc_days, '0.departures.0.trip_version.trip.code')
        base_obj['trip_version_name'] = dictor(tc_days, '0.departures.0.trip_version.version_name')
        base_obj['market'] = dictor(tc_days, '0.departures.0.trip_version.trip.seo.locale_id')
        base_obj['trip_type'] = dictor(tc_days, '0.departures.0.trip_version.trip.trip_type.name')
        base_obj['brand'] = dictor(tc_days, '0.departures.0.trip_version.trip.project.name')
        base_obj['trip_day'] = 1
    else:
        base_obj['trip_code'] = dictor(day_services, '0.trip_version_day.trip_version.trip.code')
        base_obj['trip_version_name'] = dictor(day_services, '0.trip_version_day.trip_version.version_name')
        base_obj['market'] = dictor(day_services, '0.trip_version_day.trip_version.trip.seo.locale_id')
        base_obj['trip_type'] = dictor(day_services, '0.trip_version_day.trip_version.trip.trip_type.name')
        base_obj['brand'] = dictor(day_services, '0.trip_version_day.trip_version.trip.project.name')
        base_obj['trip_day'] = dictor(day_services, '0.trip_version_day.day_number')

    rows.append(base_obj)

In [9]:
with open('mydata.csv', 'w') as f:
    f_csv = csv.DictWriter(f, headers)
    f_csv.writeheader()
    f_csv.writerows(rows)

In [10]:
df = pd.read_csv('mydata.csv')

In [11]:
df.columns

Index(['service_name', 'provider_name', 'provider_code', 'trip_code',
       'provider_type', 'service_level', 'duration_in_h', 'expense_type',
       'payment_type', 'get_feedback', 'bed_distribution', 'contract_link',
       'trip_version_name', 'market', 'trip_type', 'brand', 'trip_day',
       'currency', 'cost_pax_1', 'cost_pax_2', 'cost_pax_3', 'cost_pax_4',
       'cost_pax_5', 'cost_pax_6', 'cost_pax_7', 'cost_pax_8', 'cost_pax_9',
       'cost_pax_10', 'cost_pax_11', 'cost_pax_12', 'cost_pax_13',
       'cost_pax_14', 'cost_pax_15', 'cost_pax_16', 'cost_eur_pax_1',
       'cost_eur_pax_2', 'cost_eur_pax_3', 'cost_eur_pax_4', 'cost_eur_pax_5',
       'cost_eur_pax_6', 'cost_eur_pax_7', 'cost_eur_pax_8', 'cost_eur_pax_9',
       'cost_eur_pax_10', 'cost_eur_pax_11', 'cost_eur_pax_12',
       'cost_eur_pax_13', 'cost_eur_pax_14', 'cost_eur_pax_15',
       'cost_eur_pax_16'],
      dtype='object')

In [12]:
df.head()

,service_name,provider_name,provider_code,trip_code,provider_type,service_level,duration_in_h,expense_type,payment_type,get_feedback,...,cost_eur_pax_7,cost_eur_pax_8,cost_eur_pax_9,cost_eur_pax_10,cost_eur_pax_11,cost_eur_pax_12,cost_eur_pax_13,cost_eur_pax_14,cost_eur_pax_15,cost_eur_pax_16
0,TC costs (todo el trip) xpbccp,TC costs (Peru),NaN,ycol,TC,NaN,NaN,OTHER,NaN,NaN,...,12.441450,10.890416,9.679448,8.709015,7.921057,7.257512,6.701794,6.220725,5.806010,5.441061
1,Abraspungo - Alojamiento Driver,ALBERGUE TURISTICO ABRASPUNGO CIA. LTDA.,NaN,xvie,Accommodation,Comfort,NaN,HOTEL,BER,NaN,...,3.317720,2.903005,2.579527,2.322404,2.115047,1.932572,1.783275,1.658860,1.551034,NaN
2,Accommodation in Amphawa - Baan Mae Arom Homes...,Baan Mae Arom Homestay,NaN,xtha,Accommodation,NaN,NaN,HOTEL,BER,NaN,...,13.831600,13.831600,13.831600,13.831600,13.831600,13.831600,13.831600,13.831600,NaN,NaN
3,Accommodation in Kanchanaburi - River Kwai Jun...,River Kwai Jungle Raft,NaN,xtha,Accommodation,NaN,NaN,HOTEL,BER,NaN,...,0.027663,0.027663,0.027663,0.027663,0.027663,0.027663,0.027663,0.027663,NaN,NaN
4,Accommodation in Chiangmai - Lisu Lodge (Essen...,Lisu Lodge Chiang Mai,NaN,xtha,Accommodation,NaN,NaN,HOTEL,BER,NaN,...,0.055326,0.055326,0.055326,0.055326,0.055326,0.055326,0.055326,0.055326,NaN,NaN


In [13]:
df.to_gbq("ventura_admins.services_v1", "ventura-cluster", if_exists='replace')

1it [00:03,  3.57s/it]
